In [43]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain import FAISS
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModel, AutoModel
from datasets import Dataset
import faiss

## Read PDF and parse

In [33]:
pdf_file = "../APD_2019_report.pdf"
pdf_reader = PdfReader(pdf_file)
text = ""
for page in pdf_reader.pages[:20]:
  text += page.extract_text()

In [34]:
len(text)

68264

In [35]:
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=100,
        length_function=len
    )
chunks = text_splitter.split_text(text)
print ("Number of chunks created",len(chunks))

Number of chunks created 166


## vectorise the chunks

In [36]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [37]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [38]:
data = {
    "text": chunks
}
# Create a Hugging Face dataset
custom_dataset = Dataset.from_dict(data)
# Map the "embeddings" column
embeddings_dataset = custom_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().numpy()[0]}
)

Map: 100%|██████████| 166/166 [00:28<00:00,  5.91 examples/s]


In [52]:
import faiss

In [53]:
# embeddings_dataset.add_faiss_index(column="embeddings", index_name="faiss_index")

## Search relevant chunk with question

In [ ]:
question = "What is the proprtion of interest expense to net income in 2019 Q2?"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

In [42]:
db = FAISS.from_texts(chunks, embeddings)

NameError: name 'embeddings' is not defined

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "faiss_index", question_embedding, k=5
)

In [ ]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)